# Préambule

## Introduction & Contexte

Ce projet a été réalisé par 3 étudiants de Master 1 à l'ENSAE, dans le cadre du cours "Python for Data Science". L'idée de ce projet vient d'une intuition que nous avons eu selon laquelle il était possible de prédire le succès d'un jeu vidéo auprès des utilisateurs à partir de certaines de ses caractéristiques. L'objectif principal de ce travail est donc de tenter de vérifier ou infirmer cette intuition à l'aide de techniques statistiques de traitement de données. 

De plus, nous voulions y incorporer une partie plus originale (traitement d'images...)

# Importation des modules 

## Modules de webscrapping

On installe les packages utiles au Webscrapping des données sur les jeux

In [1]:
# On importe les packages
!pip install unicode
!pip install unidecode
import requests 
import urllib
import bs4
from requests_html import HTMLSession
from tqdm import tqdm
from unidecode import unidecode



## Importation des modules de Visualisation & Modélisation des données

In [2]:
#Là on importe les autres packages
import numpy as np
import pandas as pd

# I/ Récupération des données

## I/A Webscrapping des titres des jeux vidéos sur Wikipédia

A partir du site Wikipédia, nous allons récupérer les titres de tous les jeux vidéos sortis depuis l'année 2000. Par exemple, depuis l'URL : https://en.wikipedia.org/wiki/Category:2023_video_games nous avons accès à la liste de la quasi-totalité des jeux vidéos sortis en 2023. Nous allons donc modifier l'URL pour chaque année et constituer une liste (Liste_VG)

In [ ]:
#Url initial : https://en.wikipedia.org/wiki/2023_in_video_games
Liste_VG = []

d = {17: 11, 18 : 9, 19 : 9, 20: 12, 21: 11, 22 : 7}
for i in range(17, 23) :
    
    
    response = requests.get(url="https://en.wikipedia.org/wiki/20"+str(i)+"_in_video_games")
    soup = bs4.BeautifulSoup(response.content, 'html.parser')
    tableau_participants = soup.findAll('table', {'class' : 'wikitable'})
    for j in range(d[i], d[i]+4) :
        rows = tableau_participants[j].find_all('tr')
        first = True
        for row in rows :
            if first :
                first = False
            else:
                title = row.find('i')
                if title != None :
                    Liste_VG.append((row.find('i').text))
                
                    
print(Liste_VG)


In [4]:
print(tableau_participants[8].find_all('th'))

#ce que j'ai rajouté pour extraire les autres trucs de wikipédia mais pue un peu la merde
#col = row.find_all(['td', 'th'])
                #if len(col) >= 2 and title != None and col[1].name and col[2] != None :
                    #Liste_VG.append([title.text, col[1].text, col[2].text])


#print(platforms[0].text)

[<th>Month</th>, <th>Day</th>, <th>Title</th>, <th>Platform(s)</th>, <th>Genre(s)</th>, <th>Developer(s)</th>, <th>Publisher(s)</th>, <th><abbr title="Reference(s)">Ref.</abbr>
</th>]


## I/B Récupération des notes des jeux

L'objectif de cette partie est d'extraire du site "Métacritic", un site spécialisé qui répertorie les avis de professionnels et d'utilisateurs sur les nouvelles sorties jeux vidéos. Pour chaque jeu dans la base de données du site, une note sur 100 est attribuée (moyenne des reviews de sites spécialisés jeux vidéos) et une note sur 10 attribuée par les utilisateurs du site. Nous allons ici récupérer pour chaque jeu ses notes Métacritic qui deviendront les variables que nous tenteront de prédire par la suite. 

On remarque que l'on peut atteindre la review du jeu à partir de l'URL, il suffit d'indiquer le titre du jeu dans l'URL, en faisant attention aux espaces. 

In [5]:
#URL du site : https://www.metacritic.com/game/
#URL d'une recherche : https://www.metacritic.com/search/gran%20theft%20auto/
#URL d'une review : https://www.metacritic.com/game/grand-theft-auto-v/
#URL d'une review d'un autre site spécialisé : https://www.ign.com/games/grand-theft-auto-v

#Base URL de toutes les reviews
URL = "https://www.metacritic.com/game/"
URLbis = "https://www.metacritic.com/game/grand-theft-auto-v/"
user_agent = {'User-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/62.0.3202.94 Safari/537.36'}
site = requests.get(URLbis, user_agent)

print(site.status_code)

403


In [17]:
def recup_metascore(list) : 
    session = HTMLSession()
    liste_ratings = []

    for i in tqdm(range(len(list))) :
        #on remplace les espaces des titres par des tirets
        title = unidecode(list[i])
        title = title.lower()
        title = title.replace(':', '')
        title = title.replace('/', '-')
        title = title.replace('.', '')
        title = title.replace('&', 'and')
        title = title.replace("'", '')
        title = title.replace(' ', '-')
        Url = URL + str(title) + '/'
        #on get l'url
        DOM_Html = session.get(Url)
        chemin_metascore = DOM_Html.html.find('#__layout > div > div.c-layoutDefault_page > div.c-pageProductGame > div:nth-child(1) > div > div \
                                  > div.c-productHero_player-scoreInfo.u-grid.g-grid-container > \
                                  div.c-productHero_score-container.u-flexbox.u-flexbox-column.g-bg-white > \
                                  div.c-productHero_scoreInfo.g-inner-spacing-top-medium.g-outer-spacing-bottom-medium.g-outer-spacing-top-medium \
                                  > div:nth-child(1) > div > div.c-productScoreInfo_scoreContent.u-flexbox.u-flexbox-alignCenter.u-flexbox-justifyFlexEnd.g-width-100.u-flexbox-nowrap > \
                                  div.c-productScoreInfo_scoreNumber.u-float-right > div > div', first = True)
        
        chemin_userscore = DOM_Html.html.find('#__layout > div > div.c-layoutDefault_page > div.c-pageProductGame > div:nth-child(1) > div > div > \
                                      div.c-productHero_player-scoreInfo.u-grid.g-grid-container > div.c-productHero_score-container.u-flexbox.u-flexbox-column.g-bg-white > \
                                      div.c-productHero_scoreInfo.g-inner-spacing-top-medium.g-outer-spacing-bottom-medium.g-outer-spacing-top-medium > \
                                      div.c-productScoreInfo.u-clearfix > div.c-productScoreInfo_scoreContent.u-flexbox.u-flexbox-alignCenter.u-flexbox-justifyFlexEnd.g-width-100.u-flexbox-nowrap > \
                                      div.c-productScoreInfo_scoreNumber.u-float-right > div > div' , first = True)

        #on extrait le rating
        if chemin_metascore != None :
            metacritic_rating_inter = chemin_metascore.attrs['title']
            metacritic_rating = metacritic_rating_inter.split(' ')[1]
        else : 
            metacritic_rating = 'N/A'
        
        if chemin_userscore != None :  
            users_ratings_inter = chemin_userscore.attrs['title']
            users_ratings = float(users_ratings_inter.split(' ')[2])
        else:
            users_ratings = 'N/A'

        liste_ratings.append([list[i], metacritic_rating, users_ratings])

    return liste_ratings

    

In [13]:
texte = 'Rise & Shine'
title = unidecode(texte)
title = title.lower()
title = title.replace(':', '')
title = title.replace('/', '-')
title = title.replace('.', '')
title = title.replace('&', 'and')
title = title.replace(' ', '-')
Url = URL + str(title) + '/'
print(Url)


https://www.metacritic.com/game/rise-and-shine/


In [14]:

DOM_Html = session.get(Url)
chemin_metascore = DOM_Html.html.find('#__layout > div > div.c-layoutDefault_page > div.c-pageProductGame > div:nth-child(1) > div > div \
                                  > div.c-productHero_player-scoreInfo.u-grid.g-grid-container > \
                                  div.c-productHero_score-container.u-flexbox.u-flexbox-column.g-bg-white > \
                                  div.c-productHero_scoreInfo.g-inner-spacing-top-medium.g-outer-spacing-bottom-medium.g-outer-spacing-top-medium \
                                  > div:nth-child(1) > div > div.c-productScoreInfo_scoreContent.u-flexbox.u-flexbox-alignCenter.u-flexbox-justifyFlexEnd.g-width-100.u-flexbox-nowrap > \
                                  div.c-productScoreInfo_scoreNumber.u-float-right > div > div', first = True)

print(chemin_metascore.attrs['title'].split(' ')[1])

66


In [18]:
recup_metascore(Liste_VG[: 20])

100%|██████████| 20/20 [00:05<00:00,  3.74it/s]


[['Milkmaid of the Milky Way', '74', 6.9],
 ['Hatsune Miku: Project DIVA Future Tone', 'N/A', 7.5],
 ['Criminal Girls: Invite Only', '55', 6.8],
 ['Danganronpa V3: Killing Harmony', '81', 7.5],
 ['School Girl/Zombie Hunter', 'N/A', 'N/A'],
 ['Rise & Shine', '66', 6.2],
 ['Atelier Shallie Plus: Alchemists of the Dusk Sea', '78', 7.6],
 ['Fate/Extella: The Umbral Star', '67', 7.1],
 ['Mighty Morphin Power Rangers: Mega Battle', '49', 7.6],
 ['Birthdays the Beginning', '65', 6.5],
 ['Valkyria Revolution', '54', 5.6],
 ['Dragon Quest VIII: Journey of the Cursed King', '89', 8.7],
 ['Gravity Rush 2', '80', 8.1],
 ['Urban Empire', '62', 5.3],
 ['Don Bradman Cricket 17', '83', 7.6],
 ['Kingdom Hearts HD 2.8 Final Chapter Prologue', '76', 8.2],
 ['Pokémon Duel', '63', 5.6],
 ['Resident Evil 7: Biohazard', '86', 8.1],
 ['Tales of Berseria', '79', 7.2],
 ['Yakuza 0', '85', 8.6]]